### Import spaCy, load annotation files

In [1]:
import spacy

nlp = spacy.load("nl_core_news_lg")

In [5]:
from tqdm import tqdm

import gzip
import os

import random

import pandas

corpus = "../Corpus/"
anns = "../Annotations/"

def load(file, corpus, lines=False):
    with open(os.path.join(corpus, file), "r") as handle:
        if lines:
            return list(handle) # handle.readlines()
        else:
            return handle.read()

### Reading BRAT annotations (for select entities)

In [6]:
def read_csv(file, corpus=anns):
    file = load(file, corpus, lines=True)
    for line in file:
        tab_cols = line.split("\t")
        
        if tab_cols[0].startswith("T"):
            last = tab_cols[-1].strip()
            first = tab_cols[0]
            
            middle = tab_cols[1].split()
#             print(middle, middle[0], middle[-1])
            middle = [middle[0], middle[1], middle[-1]]
            
            yield (first, *middle, last)
            
def filter_rows(rows):
    for r in rows:
        if r[1].upper() in ["PERSON", "ORG", "GPE"]:
            yield r
            
def change_rows(rows):
    for r in rows:
        entity_nr, entity_type, start, end, label = r
        yield int(start), int(end), entity_type.upper()

### output of textfile, text and annotations - both manual and those output by spaCy 

In [7]:
data = {}
for ann_f in os.listdir(anns):
    if not ann_f.endswith('.ann'): continue
    if len(load(ann_f, anns)) > 0:
        cur_name = ann_f.strip(".ann")
    
        txt_f = cur_name + ".txt"
        if os.path.isfile(os.path.join(corpus, txt_f)):
        
            raw_text = load(txt_f, corpus)
            brat_entity_list = list(change_rows(filter_rows(read_csv(ann_f, anns))))
            spacy_entity_list = [(e.start_char, e.end_char, e.label_) for e in nlp(raw_text).ents]

            data[cur_name] = (raw_text.replace("\n", " "), brat_entity_list, spacy_entity_list)
        else:
            print("no file", txt_f)
    #else:
        #print("annotation file empty")

In [8]:
import random

def range_in(ent, ent_ls):
    for ent2 in ent_ls:
        x = ent[:-1]
        y = ent2[:-1]
        if len(range(max(x[0], y[0]), min(x[-1], y[-1])+1)) > 0:
            return True
    return False
        
def merge(b_ls, s_ls):
    l = len(b_ls)
    
    l = len(s_ls)//1
    k = 0
    for s_ent in random.sample(s_ls, len(s_ls)):
        if k == l:
            break
        if not range_in(s_ent, b_ls):
            yield s_ent
            k += 1
            
    yield from b_ls
            
    
    
    
    
data_merged = {}

for f, (txt, b_ls, s_ls) in data.items():    
    ents = sorted(merge(b_ls, s_ls), key=lambda tup: tup[0])
    
    data_merged[f] = (txt, ents)

In [9]:
data_merged


{'NL-HaNA_1.04.02_6847_0758': ("Gods dienst gedaan hebben, zal mogen verkoopen, alles ingevolge de heer gerespecteerde ordres voor de E: hooge Regeening diezertlanden ten dien opzigten g'emaneert Al het gunt voorschreeven staat de testateuren drijdelijk voorgeleesen en bi hun zoozij reiden wel verstand szijnde begeereten zij dat sent instrument het h: als testament of cod„ El, of zodanig als het best na regten sal konnen bestaan DitAleus passeerde ten huijzen ande tesluteuren Staande ter plaatse als in den Hoofde deeses gemelde in tegenswoordigheijd van Abraham Salbindusz en Pieter Iuspphoz Clercquen als getuigen De minute deeses is behoorlijk geteekend en gesch op een Legul van een rijxd:s / onderstond /quve allestar ( was geteekend) C: Blomhert nolafis Op Heeden den 15. 9br A:o 1774 Toormidagsee Ctokeuuren Gotfried hebbe ik Christiaan Baumgarten eerst gesworen Klerk van Politie deeses gouvernements, ende na te noemene getuigen als waar toe gerequireerd zijnde mij vervoegt Pasqual de 

In order to circumvent the catastrophic forgetting problem, we train the model on both manual annotations as well as what spaCy itself predicts.